In [1]:
# 导入包
import os
# 导入LangChain核心组件
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

# 步骤1：加载文档

## 1.1加载web文档

从 `load` 源码来看，返回的 `docs` 是一个 `Document` 列表，  
`Document` 有两个属性，分别是 `page_content` 和 `metadata`。

```python
def aload(self) -> List[Document]:  # type: ignore[override]
    """Load text from the urls in web_path async into Documents."""
    results = self.scrape_all(self.web_paths)
    docs = []
    for path, soup in zip(self.web_paths, results):
        text = soup.get_text(**self.bs_get_text_kwargs)
        metadata = _build_metadata(soup, path)
        docs.append(Document(page_content=text, metadata=metadata))
    return docs

In [2]:
from langchain_community.document_loaders import (
    WebBaseLoader,      # 网页
    PyPDFLoader,        # PDF
    TextLoader,         # 文本文件
    DirectoryLoader,    # 目录
    CSVLoader,          # CSV
)

loader = WebBaseLoader("https://blog.csdn.net/weixin_44919384/article/details/154616759?spm=1001.2014.3001.5501")
docs = loader.load()
print(f"加载了 {len(docs)} 个文档")
title = docs[0].metadata.get('title', 'N/A')
print(f"第一个文档的标题：{title}")
print(f"第一个文档长度: {len(docs[0].page_content)} 字符")

加载了 1 个文档
第一个文档的标题：模型训练（四）梯度累计Gradient Accumulation-CSDN博客
第一个文档长度: 11642 字符


## 1.2加载 PDF 文档

从源码来看，`lazy_load` 方法会返回一个可迭代对象（`Iterator[Document]`），每次迭代会产出（`yield`）一个 `Document` 对象。  
每当你从该迭代器中取一个元素，它就按需处理一部分内容，返回一个 `Document`，从而实现内存高效的**惰性加载**（lazy loading）。

```python
def lazy_load(self) -> Iterator[Document]:
    """
    Lazy load given path as pages.
    Insert image, if possible, between two paragraphs.
    In this way, a paragraph can be continued on the next page.
    """
    if self.web_path:
        blob = Blob.from_data(open(self.file_path, "rb").read(), path=self.web_path)
    else:
        blob = Blob.from_path(self.file_path)
    yield from self.parser.lazy_parse(blob)

In [3]:
loader = PyPDFLoader("./Dataset/PDF/基于视-触觉融合感知的机器人抓取滑动检测与力控研究_闫腾.pdf")
docs = loader.load()

# 打印方式1：
# for doc in docs:
#     print(doc.page_content)

# 打印方式2：
first_doc = docs[0]
print("meta_data:",first_doc.metadata)
print("content:",first_doc.page_content)
# second_doc = docs[1]
# print("meta_data:",first_doc.metadata)
# print("content:",first_doc.page_content)

meta_data: {'producer': 'TTKN', 'creator': 'ReaderEx_DIS 2.5.0 Build 4088', 'creationdate': '2025-11-03T14:24:27-08:00', 'author': 'CNKI', 'source': './Dataset/PDF/基于视-触觉融合感知的机器人抓取滑动检测与力控研究_闫腾.pdf', 'total_pages': 79, 'page': 0, 'page_label': '1'}
content: 硕士学位论 文
学位申请人姓名 闫腾
学位申请人学号 2200411007
专 业 名 称 机械工程
学 科 门 类 工学
学院（部、研究院） 应用技术学院
导 师 姓 名 李文贤
二〇二五年六月
分类号 学校代码 10590
UDC 密 级 公开
基于视-触觉融合感知的机器人
抓取滑动检测与力控研究
ധᎪն࿐


# 步骤2：文本分块

为什么需要分块？

- LLM上下文限制：大多数LLM有最大token限制（如GPT-4的8K/32K）

- 提高检索精度：小块文本更容易匹配特定查询

- 降低成本：只检索和处理相关部分

## 2.1创建文本分块器

In [4]:
# 创建文本分块器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 每块最大字符数
    chunk_overlap=200,      # 块之间的重叠
    length_function=len,    # 长度计算函数
    is_separator_regex=False,
)

# 分块文档
splits = text_splitter.split_documents(docs)

print(f"原始文档: {len(docs)} 个")
print(f"分块后: {len(splits)} 个")
print(f"\n第一个分块示例:\n{splits[0].page_content[:200]}...")

原始文档: 79 个
分块后: 99 个

第一个分块示例:
硕士学位论 文
学位申请人姓名 闫腾
学位申请人学号 2200411007
专 业 名 称 机械工程
学 科 门 类 工学
学院（部、研究院） 应用技术学院
导 师 姓 名 李文贤
二〇二五年六月
分类号 学校代码 10590
UDC 密 级 公开
基于视-触觉融合感知的机器人
抓取滑动检测与力控研究
ധᎪն࿐...


## 2.2分块策略对比

字符分块：简单快，但易切断句子，语义差。

递归分块：优先按段落/换行切，兼顾语义与效率，推荐默认使用。

Token分块：精准控制LLM输入长度，但依赖tokenizer，速度较慢。

语义分块：按语义边界切，质量高但计算开销大，适合高要求场景。


In [37]:
# 1. 字符分块（最基础）
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)

print(f"字符分块数量: {len(chunks)}")
print("第1块预览:\n", chunks[0].page_content[:300], "\n...")

字符分块数量: 79
第1块预览:
 硕士学位论 文
学位申请人姓名 闫腾
学位申请人学号 2200411007
专 业 名 称 机械工程
学 科 门 类 工学
学院（部、研究院） 应用技术学院
导 师 姓 名 李文贤
二〇二五年六月
分类号 学校代码 10590
UDC 密 级 公开
基于视-触觉融合感知的机器人
抓取滑动检测与力控研究
ധᎪն࿐ 
...


In [39]:
# 2. 递归分块（推荐，智能识别段落
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)
chunks = text_splitter.split_documents(docs)

print(f"递归分块数量: {len(chunks)}")
print("第1块预览:\n", chunks[0].page_content[:300], "\n...")

递归分块数量: 99
第1块预览:
 硕士学位论 文
学位申请人姓名 闫腾
学位申请人学号 2200411007
专 业 名 称 机械工程
学 科 门 类 工学
学院（部、研究院） 应用技术学院
导 师 姓 名 李文贤
二〇二五年六月
分类号 学校代码 10590
UDC 密 级 公开
基于视-触觉融合感知的机器人
抓取滑动检测与力控研究
ധᎪն࿐ 
...


In [40]:
# 3. Token分块（精确控制token数量）
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=256,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(docs)

print(f"Token分块数量: {len(chunks)}")
print("第1块预览（前100字符）:\n", chunks[0].page_content[:100], "\n...")

Token分块数量: 542
第1块预览（前100字符）:
 硕士学位论 文
学位申请人姓名 闫腾
学位申请人学号 2200411007
专 业 名 称 机械工程
学 科 门 类 工学
学院（部、研究院） 应用技术学院
导 师 姓 名 李文贤
二〇二五年六月
分 
...


In [ ]:
# 4. 语义分块（基于语义相似度）
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings

# 需设置 OPENAI_API_KEY 环境变量
embeddings = OpenAIEmbeddings()
text_splitter = SemanticChunker(embeddings)

chunks = text_splitter.split_documents(docs)

print(f"语义分块数量: {len(chunks)}")
print("第1块预览:\n", chunks[0].page_content[:300], "\n...")

# 步骤3：向量化

将文本转换为数学向量，这是语义搜索的核心。

常用的嵌入模型对比：

| 模型 | 提供商 | 维度 | 成本 | 性能 | 特点 |
|------|--------|------|------|------|------|
| text-embedding-3-small | OpenAI | 1536 | \$ | 高性价比 ⭐ | 通用场景，平衡性能 |
| text-embedding-3-large | OpenAI | 3072 | \$\$ | 最高质量 | 要求最高的应用场景 |
| text-embedding-ada-002 | OpenAI | 1536 | \$ | 上一代 | 兼容旧系统 |
| all-MiniLM-L6-v2 | HuggingFace | 384 | 免费 | 本地部署 | 轻量级，快速推理 |
| bce-embedding-base_v1 | 百度 | 768 | 免费 | 中文优化 ⭐ | 中文任务表现优异 |
| bce-reranker-base_v1 | 百度 | - | 免费 | 重排序专用 | 提升检索精度 |

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from modelscope import snapshot_download
import os

# 指定本地模型存储目录
local_models_dir = "./Models" 
os.makedirs(local_models_dir, exist_ok=True)  

# 1. 从 ModelScope 下载模型到指定目录
model_id = "maidalun/bce-embedding-base_v1"
local_model_path = snapshot_download(
    model_id,
    cache_dir=local_models_dir 
)

# 2. 创建嵌入模型（使用该本地路径）
embeddings = HuggingFaceEmbeddings(
    model_name=local_model_path,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

# 3. 测试向量化
text = "RAG是一种强大的AI技术"
vector = embeddings.embed_query(text)

print(f"文本: {text}")
print(f"向量维度: {len(vector)}")
print(f"向量前5个值: {[round(x, 4) for x in vector[:5]]}")

2025-11-11 20:03:03,135 - modelscope - INFO - PyTorch version 2.3.0+cu118 Found.
2025-11-11 20:03:03,136 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2025-11-11 20:03:03,293 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 298ceecce207285dd10b135af16e71cc and a total number of 964 components indexed
2025-11-11 20:03:04,192 - modelscope - WARNING - Model revision not specified, use revision: v0.0.3


文本: RAG是一种强大的AI技术
向量维度: 768
向量前5个值: [0.0048, 0.0216, -0.005, 0.02, -0.0113]


### 向量相似度计算

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

text1 = "苹果是一种水果"
text2 = "香蕉是一种水果"
text3 = "苹果是一种好吃的水果"

vector1 = embeddings.embed_query(text1)
vector2 = embeddings.embed_query(text2)  
vector3 = embeddings.embed_query(text3) 

# 转为 NumPy 数组并 reshape 为 (1, dim)
v1 = np.array(vector1).reshape(1, -1)
v2 = np.array(vector2).reshape(1, -1)
v3 = np.array(vector3).reshape(1, -1)

# 计算余弦相似度
sim_1_2 = cosine_similarity(v1, v2)[0][0]
sim_1_3 = cosine_similarity(v1, v3)[0][0]

print(f"「{text1}」 vs 「{text2}」 相似度: {sim_1_2:.4f}")
print(f"「{text1}」 vs 「{text3}」 相似度: {sim_1_3:.4f}")

「苹果是一种水果」 vs 「香蕉是一种水果」 相似度: 0.7481
「苹果是一种水果」 vs 「苹果是一种好吃的水果」 相似度: 0.9040


# 步骤4：存储到向量数据库

| 数据库 | 类型 | 优势 | 适用场景 |
|:-------|:----:|:-----|:---------|
| Chroma | 嵌入式 | 简单易用，无需额外服务 | 开发、小规模应用 ⭐ |
| Pinecone | 云服务 | 高性能，托管服务 | 生产环境 |
| Weaviate | 自建 | 功能丰富，开源 | 大规模部署 |
| FAISS | 库 | 速度快，Meta开源 | 研究和原型 |

将./Dataset/PDF目录下的所有PDF文件都存储到向量数据库中。

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_all_pdfs_from_directory(directory_path="./Dataset/PDF/"):
    """加载目录下所有PDF文件，并为每个分块添加唯一ID"""
    all_splits = []
    pdf_files = []
    
    # 检查目录是否存在
    if not os.path.exists(directory_path):
        print(f"❌ 目录不存在: {directory_path}")
        return all_splits, pdf_files
    
    # 获取所有PDF文件
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):
            pdf_files.append(filename)
    
    if not pdf_files:
        print(f"⚠️  目录中没有找到PDF文件: {directory_path}")
        return all_splits, pdf_files
    
    print(f"📁 找到 {len(pdf_files)} 个PDF文件:")
    
    # 创建文本分块器
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )
    
    # 全局分块计数器（用于生成全局唯一ID）
    global_chunk_index = 0
    
    # 逐个加载PDF文件
    for filename in pdf_files:
        file_path = os.path.join(directory_path, filename)
        base_name = os.path.splitext(filename)[0]  # 去掉 .pdf 后缀
        
        try:
            print(f"📖 正在加载: {filename}")
            
            # 加载PDF
            loader = PyPDFLoader(file_path)
            docs = loader.load()  # 每个 doc 对应一页，metadata 包含 page
            
            # 分块处理
            splits = text_splitter.split_documents(docs)
            
            # 为每个分块添加唯一 ID 和增强元数据
            for i, split in enumerate(splits):
                # 使用 文件名 + 页码 + 分块序号（推荐）
                page_num = split.metadata.get('page', 'unknown')
                split_id = f"{base_name}_p{page_num}_c{i}"
                 
                # 添加 ID 到 metadata
                split.metadata['id'] = split_id
                split.metadata['source_file'] = filename  # 可选：保留原始文件名
                
                all_splits.append(split)
            
            print(f"✅ {filename}: {len(docs)}页 -> {len(splits)}个分块")
            
        except Exception as e:
            print(f"❌ 加载失败 {filename}: {str(e)}")
    
    print(f"🎉 总共加载 {len(all_splits)} 个文本分块（均已添加 id）")
    return all_splits, pdf_files

# 使用示例
splits, loaded_files = load_all_pdfs_from_directory("./Dataset/PDF/")

📁 找到 10 个PDF文件:
📖 正在加载: 基于触觉感知的机器人滑动检测与抓取控制_代稷珅.pdf
✅ 基于触觉感知的机器人滑动检测与抓取控制_代稷珅.pdf: 81页 -> 1234个分块
📖 正在加载: 基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf
✅ 基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf: 182页 -> 3905个分块
📖 正在加载: 基于触觉图像序列的机器人抓取目标状态感知_韩筱.pdf


Multiple definitions in dictionary at byte 0x35029d for key /MediaBox


✅ 基于触觉图像序列的机器人抓取目标状态感知_韩筱.pdf: 99页 -> 1335个分块
📖 正在加载: 基于触视觉融合的机器人目标识别和滑动检测研究_林为梁.pdf


Multiple definitions in dictionary at byte 0x74db3a for key /MediaBox


✅ 基于触视觉融合的机器人目标识别和滑动检测研究_林为梁.pdf: 71页 -> 1074个分块
📖 正在加载: 基于视触感知协同的机器人抓取技术研究_祝会龙.pdf
✅ 基于视触感知协同的机器人抓取技术研究_祝会龙.pdf: 72页 -> 920个分块
📖 正在加载: 基于视触觉融合的机械手分类抓取方法研究_余航.pdf
✅ 基于视触觉融合的机械手分类抓取方法研究_余航.pdf: 67页 -> 780个分块
📖 正在加载: 基于视触觉融合的机器人物体识别和抓取稳定性检测的研究与应用_上官明雨.pdf
✅ 基于视触觉融合的机器人物体识别和抓取稳定性检测的研究与应用_上官明雨.pdf: 76页 -> 926个分块
📖 正在加载: 视触觉融合下的机器人抓取滑动检测研究_黄兆基.pdf
✅ 视触觉融合下的机器人抓取滑动检测研究_黄兆基.pdf: 75页 -> 562个分块
📖 正在加载: 基于视-触觉融合感知的机器人抓取滑动检测与力控研究_闫腾.pdf
✅ 基于视-触觉融合感知的机器人抓取滑动检测与力控研究_闫腾.pdf: 79页 -> 921个分块
📖 正在加载: 针对机器手稳定抓取的视触融合算法研究_闫凯波.pdf
✅ 针对机器手稳定抓取的视触融合算法研究_闫凯波.pdf: 81页 -> 1072个分块
🎉 总共加载 12729 个文本分块（均已添加 id）


在保存到向量数据库中时出现报错UnicodeEncodeError: 'utf-8' codec can't encode characters in position 795-798: surrogates not allowed，说明文档内容（splits 中的某段文本）包含非法 Unicode 字符（surrogate characters），这些字符无法被 UTF-8 编码，而 ChromaDB（底层使用 Rust）要求所有字符串必须是合法的 UTF-8。因此在保存之前需要进行清洗

In [4]:
def clean_text(text: str) -> str:
    return text.encode('utf-8', errors='ignore').decode('utf-8').strip('\n').strip()

# 清洗所有文档块的 page_content
for doc in splits:
    doc.page_content = clean_text(doc.page_content)

In [5]:
def smart_vectorstore_creation(documents, embeddings, persist_directory="./chroma_db", collection_name="langchain"):
    """智能向量数据库创建（推荐使用）"""
    
    print(f"🔧 处理集合: {collection_name}")
    
    # 确保目录存在
    os.makedirs(persist_directory, exist_ok=True)
    
    try:
        # 尝试连接到现有集合
        vectorstore = Chroma(
            collection_name=collection_name,
            persist_directory=persist_directory,
            embedding_function=embeddings
        )
        
        # 检查集合是否为空
        collection_info = vectorstore.get()
        if collection_info['ids']:
            print(f"🗑️ 清理集合中的 {len(collection_info['ids'])} 个文档")
            vectorstore.delete(ids=collection_info['ids'])
        else:
            print("✅ 集合为空，无需清理")
            
    except Exception:
        # 集合不存在，将在后续创建
        print(f"🆕 集合不存在，将创建新集合")
        vectorstore = None
    
    # 分批添加文档
    total_docs = len(documents)
    batch_size = 1000
    
    for i in range(0, total_docs, batch_size):
        batch = documents[i:i + batch_size]
        batch_num = i // batch_size + 1
        total_batches = (total_docs - 1) // batch_size + 1
        
        print(f"📦 添加批次 {batch_num}/{total_batches}: {len(batch)} 个文档")
        
        if vectorstore is None and i == 0:
            # 第一批且集合不存在，创建新集合
            vectorstore = Chroma.from_documents(
                documents=batch,
                embedding=embeddings,
                persist_directory=persist_directory,
                collection_name=collection_name
            )
        else:
            # 后续批次或集合已存在，添加到现有集合
            vectorstore.add_documents(batch)
    
    print(f"✅ 完成！集合 '{collection_name}' 现有 {total_docs} 个文档")
    return vectorstore



In [6]:
vectorstore = smart_vectorstore_creation(splits, embeddings, collection_name="langchain")

🔧 处理集合: langchain
✅ 集合为空，无需清理
📦 添加批次 1/13: 1000 个文档
📦 添加批次 2/13: 1000 个文档
📦 添加批次 3/13: 1000 个文档
📦 添加批次 4/13: 1000 个文档
📦 添加批次 5/13: 1000 个文档
📦 添加批次 6/13: 1000 个文档
📦 添加批次 7/13: 1000 个文档
📦 添加批次 8/13: 1000 个文档
📦 添加批次 9/13: 1000 个文档
📦 添加批次 10/13: 1000 个文档
📦 添加批次 11/13: 1000 个文档
📦 添加批次 12/13: 1000 个文档
📦 添加批次 13/13: 729 个文档
✅ 完成！集合 'langchain' 现有 12729 个文档


# 步骤5：检索

检索类型：

1. 相似度搜索 (Similarity Search)：最基础的搜索方式，返回与查询最相似的文档。

```python
retriever = vectorstore.as_retriever(
search_type="similarity",
search_kwargs={"k": 5} # 返回 top-5
)
```

2. 最大边际相关性 (MMR)：平衡相关性和多样性，避免返回过于相似的结果。

```python
retriever = vectorstore.as_retriever(
search_type="mmr",
search_kwargs={
"k": 5,
"fetch_k": 20, # 先获取 20 个候选
"lambda_mult": 0.5 # 多样性参数 (0=最多样, 1=最相关)
}
)
```

    MMR 工作原理：
    
    查询: "机器学习算法"
    
    步骤1: 获取 top-20 相似文档
        1. "深度学习是机器学习的一个分支..." (相似度: 0.95)
        2. "深度学习使用神经网络..." (相似度: 0.94) ← 与第1个很相似
        3. "决策树是一种机器学习算法..." (相似度: 0.90)
        4. "支持向量机(SVM)用于分类..." (相似度: 0.88)
    ...
    
    步骤2: MMR 选择 (k=3, lambda=0.5)
    选中: #1 (最相关)
    选中: #3 (相关且与 #1 不同) ← 跳过 #2 因为太相似
    选中: #4 (增加多样性)

3. 相似度阈值 (Similarity Score Threshold)：只返回相似度超过阈值的文档。

```python
retriever = vectorstore.as_retriever(
search_type="similarity_score_threshold",
search_kwargs={
"score_threshold": 0.5, # 只返回相似度 > 0.5 的结果
"k": 5
}
)
```


| 策略 | 优势 | 劣势 | 适用场景 |
|------|------|------|----------|
| Similarity | 简单快速 | 可能重复 | 默认选择 |
| MMR | 结果多样 | 稍慢 | 需要不同角度信息 |
| Threshold | 质量保证 | 可能无结果 | 严格匹配需求 |

检索的结果也会出现文本包含大量乱码/排版符号（如 , , 换行混乱等），这是 PDF 解析时常见的问题，所以也需要进行文本清理。

In [8]:
import re

def clean_text(text: str) -> str:
    """
    移除所有中文字符之间的任意空白字符（包括普通空格、全角空格、不间断空格等）
    同时保留英文/数字间的正常空格。
    """
    # 匹配：中文 + (任意空白序列) + 中文 → 替换为两个中文直接相连
    # \u4e00-\u9fff: 基本汉字
    # \u3400-\u4dbf: 扩展A
    # \u20000-\u2a6df: 扩展B（但Python re不支持4字节，可忽略或用代理）
    chinese_char = r'[\u4e00-\u9fff]'
    # 匹配任意空白（包括全角、不间断、制表符等）
    any_whitespace = r'[\s\u00A0\u2000-\u200F\u2028-\u202F\u3000]+'
    
    pattern = f'({chinese_char}){any_whitespace}(?={chinese_char})'
    # 使用正向先行断言，避免吃掉后面的中文
    result = re.sub(pattern, r'\1', text)
    
    # 额外：清理连续多余空格（非中文区域）
    result = re.sub(r'\s+', ' ', result)
    return result.strip()

# 检查向量数据库是否存在
if os.path.exists("./chroma_db"):
    # 加载现有向量数据库
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )
    
    collection = vectorstore._collection
    doc_count = collection.count()
    
    print(f"✅ 向量数据库加载成功！")
    print(f"   已存在 {doc_count} 个文档块")
    print(f"   加载路径: ./chroma_db")
    
    collection_name = collection.name
    metadata = collection.metadata or {}
    distance_metric = metadata.get("hnsw:space", "未指定（默认通常为 cosine）")
    
    print(f"   Collection 名称: {collection_name}")
    print(f"   相似度度量方式: {distance_metric}")
    
retriever = vectorstore.as_retriever(
search_type="similarity",
search_kwargs={"k": 5} # 返回 top-5
)

✅ 向量数据库加载成功！
   已存在 12721 个文档块
   加载路径: ./chroma_db
   Collection 名称: langchain
   相似度度量方式: 未指定（默认通常为 cosine）


**向量数据库与Collection的关系：**

| 向量数据库概念 | 传统关系型数据库（如 MySQL）类比 |
|:--------------|:--------------------------------|
| 向量数据库（Vector Database） | 整个 数据库实例（Database Instance），比如 my_company_db |
| Collection（集合） | 一个 数据表（Table），比如 users 表、products 表 |
| 一条文档 + 嵌入向量 | 表中的一行记录（Row / Record） |
| 嵌入维度、相似度度量方式 | 表的结构定义（Schema），比如字段类型、索引方式 |

In [7]:
# 执行检索
query = "什么是视触觉？"
docs = retriever.invoke(query)

# 查看结果
for i, doc in enumerate(docs, 1):
    cleaned_content = clean_text(doc.page_content)  # 清理前300字符
    source = doc.metadata.get('source', 'N/A')
    id = doc.metadata.get('id', 'N/A')
    print(f"\n📄 结果 {i}:")
    print(f"内容: {cleaned_content}")
    print(f"来源: {source}")
    print(f"id：{id}")


📄 结果 1:
内容: perception 皮肤。使用一种被称为Gelsight的触觉传感器作为机器人的指尖触觉感受器，该传感器可以测量高分辨率的几何形状并以 3D方式重建接触面，此外可用于间接测量法向
来源: ./Dataset/PDF/基于触觉图像序列的机器人抓取目标状态感知_韩筱.pdf
id：基于触觉图像序列的机器人抓取目标状态感知_韩筱_p23_c383

📄 结果 2:
内容: 1.2.2 触觉感知 ..................................................................................... 3
来源: ./Dataset/PDF/基于视触觉融合的机械手分类抓取方法研究_余航.pdf
id：基于视触觉融合的机械手分类抓取方法研究_余航_p5_c61

📄 结果 3:
内容: 觉传感器 ， 最主要的测量对象即是与目标物的相互作用力。 由于 ， 电子皮肤的实质是高密度的阵列式传感器 ， 其采集到的触觉信号是与传
来源: ./Dataset/PDF/基于触觉感知的机器人滑动检测与抓取控制_代稷珅.pdf
id：基于触觉感知的机器人滑动检测与抓取控制_代稷珅_p39_c535

📄 结果 4:
内容: 1.2.3 视触觉融合感知人们对一个物体的描述往往是从多个角度去描述的，依靠的是多个器官的共同感知，其中最重要的是视觉感知和触觉感知，视觉感知提供物体的颜色、形状
来源: ./Dataset/PDF/基于视触觉融合的机械手分类抓取方法研究_余航.pdf
id：基于视触觉融合的机械手分类抓取方法研究_余航_p11_c178

📄 结果 5:
内容: 反映 综上 ， 人手具有极高的灵活性和丰富的触觉感知能力， 能够灵活 、 可靠地抓 握和操作各种物体 。 通过融合触觉 、 视
来源: ./Dataset/PDF/基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf
id：基于视触感知协同的多指灵巧手抓取方法研究_张静_p33_c685


In [15]:
# 执行检索
query = "什么是抓取检测？"
docs = retriever.invoke(query)

# 查看结果
for i, doc in enumerate(docs, 1):
    cleaned_content = clean_text(doc.page_content)  # 清理前300字符
    source = doc.metadata.get('source', 'N/A')
    id = doc.metadata.get('id', 'N/A')
    print(f"\n📄 结果 {i}:")
    print(f"内容: {cleaned_content}")
    print(f"来源: {source}")
    print(f"id：{id}")


📄 结果 1:
内容: 人灵巧抓取过程 ， 使机器人能够根据抓取过程中物理交互情况 ， 及时调整抓取姿态和夹持力度 ， 是 保证机器人采摘过程
来源: ./Dataset/PDF/基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf
id：基于视触感知协同的多指灵巧手抓取方法研究_张静_p164_c3266

📄 结果 2:
内容: 西南科技大学硕士学位论文 32 图 4-2 抓取过程 Fig.4-2 Grabbing Process （1）抓手打开阶段：抓手在初始位置将抓取打开到最大开合的状态， 等待抓
来源: ./Dataset/PDF/基于视触感知协同的机器人抓取技术研究_祝会龙.pdf
id：基于视触感知协同的机器人抓取技术研究_祝会龙_p39_c500

📄 结果 3:
内容: 免目标物发生滑动。 根据滑动检测算 法的结果， 对夹取力作出相对应的调整是抓取的基本控制策略。 研宄人员近年来 在滑动检测及实现抓取
来源: ./Dataset/PDF/基于触觉感知的机器人滑动检测与抓取控制_代稷珅.pdf
id：基于触觉感知的机器人滑动检测与抓取控制_代稷珅_p17_c212

📄 结果 4:
内容: 抓取功能， 使能机器人视触协同抓取优化算法 ， 观察机器人在 接触物体后， 尝试夹持的过程 。 记录机器人在尝试夹持至夹
来源: ./Dataset/PDF/基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf
id：基于视触感知协同的多指灵巧手抓取方法研究_张静_p154_c3054

📄 结果 5:
内容: 体进行抓取。 触觉感知：抓取过程中使用触觉传感器对物体进行触觉感知，感知物体硬度
来源: ./Dataset/PDF/基于视触感知协同的机器人抓取技术研究_祝会龙.pdf
id：基于视触感知协同的机器人抓取技术研究_祝会龙_p18_c277


In [18]:
# 执行检索
query = "什么是机械臂？"
docs = retriever.invoke(query)

# 查看结果
for i, doc in enumerate(docs, 1):
    cleaned_content = clean_text(doc.page_content)  # 清理前300字符
    source = doc.metadata.get('source', 'N/A')
    id = doc.metadata.get('id', 'N/A')
    print(f"\n📄 结果 {i}:")
    print(f"内容: {cleaned_content}")
    print(f"来源: {source}")
    print(f"id：{id}")


📄 结果 1:
内容: 尽管， 机械 臂的运动速度可以设定为匀速运动， 避免加速度变化而造成对目标和夹爪之间动 力学平衡关系的影响 。 但是 ， 机械臂的
来源: ./Dataset/PDF/基于触觉感知的机器人滑动检测与抓取控制_代稷珅.pdf
id：基于触觉感知的机器人滑动检测与抓取控制_代稷珅_p66_c947

📄 结果 2:
内容: ±180°/s，机械臂末端速度视各关节载荷与实际速度而定。机械臂本体的位姿可重复性精度为 ±0.03mm。与其它型号机械臂不同的是，UR5e机械臂的工具法兰带有力感应
来源: ./Dataset/PDF/基于触觉图像序列的机器人抓取目标状态感知_韩筱.pdf
id：基于触觉图像序列的机器人抓取目标状态感知_韩筱_p25_c407

📄 结果 3:
内容: 机械臂可重复性 +-0.03mm 有效负载 5 千克/11 磅工作半径 850mm/33.5 英寸自由度 6 个旋转关节
来源: ./Dataset/PDF/基于视触感知协同的机器人抓取技术研究_祝会龙.pdf
id：基于视触感知协同的机器人抓取技术研究_祝会龙_p19_c282

📄 结果 4:
内容:  ０ ． ０８２３  ０  本文将冗余度机械臂运动控制指标定义为 ＆ 代表着机械臂的重复运动性能 ， 其表达式可以记为
来源: ./Dataset/PDF/基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf
id：基于视触感知协同的多指灵巧手抓取方法研究_张静_p138_c2748

📄 结果 5:
内容:  第５ 章机械手稳定抓取中的力位分层控制  ｒ  Ｖ  ｆ  ｆ   ＾  ｒ （ ｂ） 机械臂沿圆周运动 ， ： ： 人 Ｍ ｉ： 一卿
来源: ./Dataset/PDF/基于视触感知协同的多指灵巧手抓取方法研究_张静.pdf
id：基于视触感知协同的多指灵巧手抓取方法研究_张静_p147_c2919


## 5.1高级检索：自定义检索器

| 步骤 | 操作 | 目的 |
|------|------|------|
| 1️⃣ | 扩大检索范围（k=10） | 获取更多候选，避免遗漏 |
| 2️⃣ | 按元数据过滤（如 type='article'） | 排除目录、图注等无关内容 |
| 3️⃣ | 重排序（用更精细的打分函数） | 提升 top 结果的相关性 |
| 4️⃣ | 返回 top-5 | 给下游（如 LLM）提供高质量上下文 |

In [12]:
from langchain_core.runnables import RunnablePassthrough

def custom_retriever(question: str) -> list:
    """自定义检索逻辑"""
    
    # 1. 基础检索
    base_docs = vectorstore.similarity_search(question, k=10)
    
    # 2. 根据元数据过滤
    filtered_docs = [
        doc for doc in base_docs
        if "张静" in doc.metadata.get('source')
    ]
    
    # 3. 重排序（可以使用更复杂的模型）
    scored_docs = [
        (doc, calculate_relevance_score(doc, question))
        for doc in filtered_docs
    ]
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    
    # 4. 返回top-k
    return [doc for doc, score in scored_docs[:5]]

## 5.2评估检索质量

In [19]:
def evaluate_retrieval(retriever, test_cases):
    """评估检索器性能"""
    
    metrics = {
        "precision": [],  # 精确率
        "recall": [],     # 召回率
    }
    
    for query, expected_doc_ids in test_cases:
        # 执行检索
        retrieved_docs = retriever.invoke(query)
        retrieved_ids = [doc.metadata['id'] for doc in retrieved_docs]
        
        # 计算指标
        relevant_retrieved = set(retrieved_ids) & set(expected_doc_ids)
        
        precision = len(relevant_retrieved) / len(retrieved_ids) if retrieved_ids else 0
        recall = len(relevant_retrieved) / len(expected_doc_ids) if expected_doc_ids else 0
        
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
    
    return {
        "avg_precision": sum(metrics["precision"]) / len(metrics["precision"]),
        "avg_recall": sum(metrics["recall"]) / len(metrics["recall"])
    }

# 使用示例
test_cases = [
    ("什么是抓取检测？", ["基于视触感知协同的多指灵巧手抓取方法研究_张静_p164_c3266", 
                  "基于视触感知协同的机器人抓取技术研究_祝会龙_p39_c500", 
                  "基于触觉感知的机器人滑动检测与抓取控制_代稷珅_p17_c212",
                  "基于触觉图像序列的机器人抓取目标状态感知_韩筱_p23_c383"]),
    ("什么是机械臂？", ["基于触觉图像序列的机器人抓取目标状态感知_韩筱_p25_c407"]),
]

results = evaluate_retrieval(retriever, test_cases)
print(f"平均精确率: {results['avg_precision']:.2%}")
print(f"平均召回率: {results['avg_recall']:.2%}")

平均精确率: 40.00%
平均召回率: 87.50%


# 步骤6：生成

基于检索到的文档，让LLM生成答案。

In [2]:
import os
import chromadb
from vllm import LLM, SamplingParams
from modelscope import snapshot_download
from langchain_community.embeddings import HuggingFaceEmbeddings

class LocalQASystem:
    """本地问答系统"""
    
    def __init__(self, model_dir, chroma_db_path="./chroma_db", embeddings_model_path=None):
        """
        初始化问答系统
        
        Args:
            model_dir: 对话模型路径
            chroma_db_path: Chroma数据库路径
            embeddings_model_path: 嵌入模型路径（与创建时相同）
        """
        self.model_dir = self._setup_model_dir(model_dir)
        self.chroma_db_path = chroma_db_path
        # 设置嵌入模型路径
        if embeddings_model_path:
            self.embeddings_model_path = embeddings_model_path
        else:
            self.embeddings_model_path = "./Models/maidalun/bce-embedding-base_v1"
        self._setup_model()
        self._setup_exact_embeddings() 
        self._setup_vectorstore()
        print("✅ 问答系统初始化完成")
    
    def _setup_model_dir(self, model_dir):
        if os.path.exists(model_dir):
            print(f"📁 使用本地模型: {model_dir}")
            return model_dir
        else:
            print(f"⬇️  从ModelScope下载模型: {model_dir}")
            return snapshot_download(model_dir)
    
    def _setup_model(self):
        os.environ['VLLM_USE_MODELSCOPE'] = 'True'
        self.llm = LLM(
            model=self.model_dir,
            tokenizer=self.model_dir,
            trust_remote_code=True
        )
        print("🤖 vLLM模型初始化完成")
    
    def _setup_exact_embeddings(self):
        """使用与创建时完全相同的嵌入模型"""
        if self.embeddings_model_path:
            # 使用您指定的路径
            self.embeddings = HuggingFaceEmbeddings(
                model_name=self.embeddings_model_path,
                model_kwargs={"device": "cuda"},
                encode_kwargs={"normalize_embeddings": True}
            )
            print(f"✅ 使用嵌入模型: {self.embeddings_model_path}")
        else:
            # 使用与创建时相同的模型ID
            self.embeddings = HuggingFaceEmbeddings(
                model_name="maidalun/bce-embedding-base_v1",
                model_kwargs={"device": "cuda"},
                encode_kwargs={"normalize_embeddings": True}
            )
            print("✅ 使用默认嵌入模型: maidalun/bce-embedding-base_v1")
        
        # 测试嵌入维度
        test_vector = self.embeddings.embed_query("test")
        print(f"🔢 嵌入模型维度: {len(test_vector)}")
    
    def _setup_vectorstore(self):
        """初始化向量数据库"""
        try:
            self.client = chromadb.PersistentClient(path=self.chroma_db_path)
            self.collection = self.client.get_collection("langchain")
            print(f"🗂️  向量数据库加载成功: {self.chroma_db_path}")
            
            # 检查集合状态
            count = self.collection.count()
            print(f"📄 文档数量: {count}")
            
        except Exception as e:
            print(f"❌ 向量数据库加载失败: {e}")
            self.collection = None
    
    def retrieve_with_exact_embedding(self, query, n_results=5):
        """使用精确匹配的嵌入模型进行检索"""
        if not self.collection:
            return {"documents": [[]]}
        
        try:
            # 关键：使用与创建时相同的嵌入模型生成查询向量
            query_embedding = self.embeddings.embed_query(query)
            print(f"🔍 查询嵌入维度: {len(query_embedding)}")
            
            results = self.collection.query(
                query_embeddings=[query_embedding],  # 使用嵌入向量
                n_results=n_results
            )
            print(f"✅ 检索到 {len(results['documents'][0])} 个相关文档")
            return results
            
        except Exception as e:
            print(f"❌ 检索失败: {e}")
            return {"documents": [[]]}
    
    def ask(self, question, max_tokens=512, temperature=0.3, n_sources=5):
        """问答函数"""
        if not self.collection:
            return {
                "question": question,
                "answer": "❌ 向量数据库不可用",
                "sources": 0,
                "error": "Vector database not available"
            }
        
        # 1. 检索相关文档（使用精确匹配的嵌入模型）
        results = self.retrieve_with_exact_embedding(question, n_sources)
        
        if not results or not results.get('documents') or not results['documents'][0]:
            return {
                "question": question,
                "answer": "❌ 未找到相关文档",
                "sources": 0,
                "error": "No relevant documents found"
            }
        
        # 2. 构建上下文
        documents = results['documents'][0]
        context = "\n\n".join(documents)
        
        # 3. 构建提示词
        prompt = self._build_prompt(question, context)
        # print("="*50)
        # print("生成的prompt：\n")
        # print(prompt)
        # print("="*50)
        
        # 4. 生成答案
        answer = self._generate_answer(prompt, max_tokens, temperature)
        
        return {
            "question": question,
            "answer": answer.strip(),
            "sources": len(documents),
            "context": context,
            "documents": documents
        }
        
    def _clean_context(self, text):
        """清洗单个文档内容"""
        import re
        if not text or not isinstance(text, str):
            return text
        
        # 1. 合并被换行符错误分割的文字（如：\n运\n动\n → 运动）
        # 匹配：换行符前后都是非空白字符（尤其是中文、字母、数字），则去掉换行
        cleaned = re.sub(r'(?<=[^\s])\n(?=[^\s])', '', text)
        # 2. 处理您提到的特定空格模式：\n            \n
        cleaned = re.sub(r'\n\s+\n', '\n\n', cleaned)  # 多行空白压缩为单个空行
        cleaned = re.sub(r'\n\s+', '\n', cleaned)     # 行内多余空格
        # 3. 处理您提到的特定模式：指\n            \n
        cleaned = re.sub(r'([\u4e00-\u9fa5a-zA-Z0-9])\n\s*\n', r'\n\n', cleaned)
        # 4. 移除多余的空行（保留最多一个空行）
        cleaned = re.sub(r'\n\s*\n', '\n\n', cleaned)
        # 5. 清理行首行尾空格
        cleaned = re.sub(r'^\s+|\s+$', '', cleaned, flags=re.MULTILINE)
        # 6. 清理连续空格
        cleaned = re.sub(r'[ \t]{2,}', ' ', cleaned)
        
        return cleaned.strip()
        
    def _build_prompt(self, question, context):
        """ChatML提示词"""
        cleaned_context = self._clean_context(context)
        
        return f"""<|im_start|>system
            你是一个专业的AI助手。请严格基于以下上下文信息回答问题：
            
            {cleaned_context}
            
            请遵循以下规则：
            1. 只使用上下文中的信息回答
            2. 如果上下文不包含相关信息，请回答"我不知道"
            3. 保持回答准确、简洁
            4. 不要编造信息<|im_end|>
            <|im_start|>user
            {question}<|im_end|>
            <|im_start|>assistant
            """
    
    def _generate_answer(self, prompt, max_tokens, temperature):
        """生成答案"""
        sampling_params = SamplingParams(
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        # print("outputs:",outputs)
        return outputs[0].outputs[0].text


/root/shared-nvme/conda-envs/py310/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025-11-10 23:27:46,884 - modelscope - INFO - PyTorch version 2.3.0+cu118 Found.
2025-11-10 23:27:46,886 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2025-11-10 23:27:46,921 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 298ceecce207285dd10b135af16e71cc and a total number of 964 components indexed


In [3]:
# 初始化问答系统
qa_system = LocalQASystem(
    model_dir="../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8",
    chroma_db_path="./chroma_db",
)

📁 使用本地模型: ../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8
INFO 11-10 23:27:47 config.py:177] The model is convertible to Marlin format. Using Marlin kernel.
INFO 11-10 23:27:47 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8', speculative_config=None, tokenizer='../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8)
WARNING 11-10 23:27:47 tokenizer.py:126] Using a slow tokenizer. This might cause a significant slowdown. Consider 

In [6]:
# 测试问答
questions = [
    "什么是机械臂？它有什么能力？",
    "解释一下滑动检测的基本概念",
]

print("🚀 开始测试问答系统")
print("=" * 70)

for i, question in enumerate(questions, 1):
    print(f"\n🎯 问题 {i}: {question}")
    print("-" * 50)
    
    result = qa_system.ask(question, max_tokens=400, temperature=0.3)
    # print("result:",result)
    print(f"💡 答案: {result['answer']}")
    print(f"📚 参考文档: {result['sources']} 个")
    
    # 显示文档预览
    # for j, doc in enumerate(result.get('documents', [])[:2], 1):
    #     preview = doc[:150] + "..." if len(doc) > 150 else doc
    #     print(f"   📄 文档 {j}: {preview}")

🚀 开始测试问答系统

🎯 问题 1: 什么是机械臂？它有什么能力？
--------------------------------------------------
🔍 查询嵌入维度: 768
✅ 检索到 5 个相关文档


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


💡 答案: 机械臂是一种可以自动执行任务的机器人手臂，它具有精确度高、可重复性好、负载能力强、工作半径大、自由度多等能力。
📚 参考文档: 5 个

🎯 问题 2: 解释一下滑动检测的基本概念
--------------------------------------------------
🔍 查询嵌入维度: 768
✅ 检索到 5 个相关文档


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

💡 答案: 滑动检测是一种用于检测物体是否发生滑动的算法。它通过比较帧间的标准差来判断物体是否发生滑动。如果帧间的标准差超过设定的阈值，那么就认为物体发生了滑动。滑动检测算法通常用于机械臂的抓取任务中，以防止物体在抓取过程中滑落。
📚 参考文档: 5 个


### Chain Types详解

LangChain提供了4种不同的链类型来处理检索到的文档：

1. Stuff（填充） - 最简单，推荐优先使用

```python
chain_type="stuff"

# 工作原理：
提示词: """
基于以下文档回答问题：

文档1: ...
文档2: ...  
文档3: ...

问题: {{question}}
答案:
"""
```

优点：简单直接，只需要一次LLM调用，成本低。 缺点：文档过多会超过上下文限制

2. Map-Reduce（映射-归约） - 处理大量文档

```python
chain_type="map_reduce"

# 工作原理：
# Map阶段：对每个文档分别提问
文档1 → LLM → 部分答案1
文档2 → LLM → 部分答案2
文档3 → LLM → 部分答案3

# Reduce阶段：合并所有部分答案
[部分答案1, 部分答案2, 部分答案3] → LLM → 最终答案
```

优点：可以处理任意数量文档，可以并行处理。 缺点：需要多次LLM调用（成本高），可能丢失文档间的关联。

3. Refine（精炼） - 迭代改进答案

```python
chain_type="refine"

# 工作原理：
文档1 → LLM → 初始答案
[初始答案 + 文档2] → LLM → 改进答案1
[改进答案1 + 文档3] → LLM → 最终答案
```

优点：答案质量高，保持文档间的关联。 缺点：顺序敏感（文档顺序影响结果），不能并行。

4.  Map-Rerank（映射-重排序） - 找最佳答案

```python
chain_type="map_rerank"

# 工作原理：
文档1 → LLM → 答案1 (置信度: 0.8)
文档2 → LLM → 答案2 (置信度: 0.6)
文档3 → LLM → 答案3 (置信度: 0.9) ← 选择这个
```

5.  Chain Type选择指南

| 场景 | 推荐类型 | 原因 |
|:-----|:--------:|:-----|
| 文档少(<10) | Stuff | 最简单高效 |
| 文档多(>10) | Map-Reduce | 可扩展 |
| 需要高质量答案 | Refine | 迭代改进 |
| 需要最相关答案 | Map-Rerank | 自动选择最佳 |